In [1]:
import plotly.express as px
import json
import os
import pandas as pd

In [2]:
import plotly.io as pio
pio.renderers.default = "browser"

In [3]:
path = 'C:/Users/WB585318/WBG/Javier Romero - Panama/Pre-SCD/'
data = path+'02 data/maps'
os.chdir(data)



print(data)

C:/Users/WB585318/WBG/Javier Romero - Panama/Pre-SCD/02 data/maps


In [26]:
panama_corr = json.load(open("PAN_corr.geojson","r",encoding='ISO-8859-1'))

In [38]:
panama_distr = json.load(open("distritos-panama.geojson","r",encoding='utf8'))

In [5]:
### creo id por corregimientos 
corr_id_map = {}
for feature in panama_corr['features']:
    feature['ids'] = feature['properties']['Codigo_u']
    corr_id_map[feature['properties']['Nombre']] = feature['ids']

In [190]:
## creo diccionario con Ids de los corregimientos
corr_id_map_1 = {}
for geometries in panama_corr['objects']['collection']['geometries']:
    geometries['ids'] = geometries['properties']['COD_CORR']
    corr_id_map_1[geometries['properties']['CORR_NOMB']] = geometries['ids']

In [39]:
panama_distr['features'][0]['properties']

{'OBJECTID': 1,
 'PROV_ID': '01',
 'DIST_ID': '02',
 'DIST_NOMBR': 'CHANGUINOLA',
 'DIST_CODIG': '0102',
 'LABEL': 'Changuinola',
 'Shape_Leng': 499296.123006,
 'Shape_Area': 4000131127.91}

In [40]:
### creo diccionario por distritos
distr_id_map = {}
for feature in panama_distr['features']:
    feature['ids'] = feature['properties']['DIST_CODIG']
    distr_id_map[feature['properties']['DIST_NOMBR']] = feature['ids']

In [48]:

df = pd.read_csv("Povertymap_2011_2015.csv")
df['ids'] = df['corr_nom'].apply(lambda x:corr_id_map[x]) 
df['extreme_pov'] = df['av_FGT0e_15']
df['poverty'] = df['av_FGT0g_15']

In [52]:
### creo id por distritos
df1 = pd.read_csv("Povertymap_2015_distr.csv")
df1['ids'] = df1['dist_nom'].apply(lambda x:distr_id_map[x]) 
df1['extreme_pov'] = df1['e_poor_dist_15']
df1['poverty'] = df1['g_poor_dist_15']

In [53]:
df1.head()

,codigo,prov,dist,corr,prov_nom,dist_nom,corr_nom,nivel,poblac_15,poor_e_15,mean_poor_e_15,mean_people_dis_15,poor_g_15,mean_poor_g_15,e_poor_dist_15,g_poor_dist_15,ids,extreme_pov,poverty
0,20101,2,1,1,COCLE,AGUADULCE (CAB),AGUADULCE (AGUADULCE),TOTAL CORREGIMIENTO,8476,106.6699,305.9147,8586.800,498.6984,1244.0340,0.035626,0.144877,0201,0.035626,0.144877
1,40101,4,1,1,CHIRIQUI,ALANJE,ALANJE (CABECERA),TOTAL CORREGIMIENTO,2381,183.6249,280.4305,1830.333,557.6128,700.4833,0.153213,0.382708,0401,0.153213,0.382708
2,20201,2,2,1,COCLE,ANTÓN,ANTON (CABECERA),TOTAL CORREGIMIENTO,9767,477.8510,588.4634,5242.600,1625.7410,1598.9480,0.112246,0.304991,0202,0.112246,0.304991
3,130101,13,1,1,PANAMA OESTE,ARRAIJÁN,ARRAIJAN (CABECERA),TOTAL CORREGIMIENTO,40881,752.3994,380.9505,27457.000,3326.2520,1738.3160,0.013874,0.063311,0801,0.013874,0.063311
4,90101,9,1,1,VERAGUAS,ATALAYA,ATALAYA (CABECERA),TOTAL CORREGIMIENTO,4888,197.9118,106.1488,2002.800,768.6484,394.9706,0.053000,0.197209,0901,0.053000,0.197209


In [42]:
## poverty by corregimientos
fig = px.choropleth(df,locations='ids',geojson=panama_corr, featureidkey='properties.Codigo_u', color='poverty')
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [63]:
## poverty by districts 

fig = px.choropleth(df1,locations='ids',
                    geojson=panama_distr, 
                    featureidkey='properties.DIST_CODIG', 
                    color='poverty',
                    hover_name='dist_nom',
                   color_continuous_scale="Viridis",
                   labels='dist_nom')
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [62]:
## extreme poverty by districts 

fig = px.choropleth(df1,locations='ids',
                    geojson=panama_distr, 
                    featureidkey='properties.DIST_CODIG', 
                    color='extreme_pov',
                    hover_name='dist_nom',
                   color_continuous_scale="Viridis",
                   labels='dist_nom')
fig.update_geos(fitbounds="locations", visible=False)
fig.update_traces(colorbar_orientation='h')
fig.show()